## Task 3: Warping images by composing rigid transformations

Implement a class RigidTransform which should specify a 3D rigid transformation which can wrap 3D image volumes. All units should be in voxel

- A class __init__ function which takes a set of 6 rigid transformations parameters, as input. Precompute a rotation matrix and a translation vector and stored in the returned class object.

In [1]:
import numpy as np

In [3]:
class RigidTransform:

    # Initialize class
    def __init__(self, rotations, translations):
        self.rx = rotations[0]
        self.ry = rotations[1]
        self.rz = rotations[2]
        self.tx = translations[0]
        self.ty = translations[1]
        self.tz = translations[2]

        # Precompute rotation matrices around the X, Y, and Z axis
        rot_x = np.array([[1, 0, 0],
                                 [0, np.cos(self.rx), -np.sin(self.rx)],
                                 [0, np.sin(self.rx), np.cos(self.rx)]])
                                 
        rot_y = np.array([[np.cos(self.ry), 0, np.sin(self.ry)],
                                [0, 1, 0],
                                [-np.sin(self.ry), 0, np.cos(self.ry)]])

        rot_z = np.array([[np.cos(self.rz), -np.sin(self.rz), 0],
                                [np.sin(self.rz), np.cos(self.rz), 0],
                                [0, 0, 1]])

        # Compose rotation matrix from yaw, pitch, and roll 
        # Correct order is Z, Y, X
        self.rot_vec = np.matmul(rot_z, np.matmul(rot_y, rot_x))

        # Compose translation vector
        self.trans_vec = np.array([self.tx, self.ty, self.tz])
        
        

In [ ]:
import numpy as np

class RigidTransform:
    def __init__(self, rx, ry, rz, tx, ty, tz):
        self.rx = rx
        self.ry = ry
        self.rz = rz
        self.tx = tx
        self.ty = ty
        self.tz = tz
        
        # Precompute rotation matrix
        R_x = np.array([[1, 0, 0],
                        [0, np.cos(rx), -np.sin(rx)],
                        [0, np.sin(rx), np.cos(rx)]])
        R_y = np.array([[np.cos(ry), 0, np.sin(ry)],
                        [0, 1, 0],
                        [-np.sin(ry), 0, np.cos(ry)]])
        R_z = np.array([[np.cos(rz), -np.sin(rz), 0],
                        [np.sin(rz), np.cos(rz), 0],
                        [0, 0, 1]])
        self.R = np.matmul(R_z, np.matmul(R_y, R_x))
        
        # Precompute translation vector
        self.T = np.array([tx, ty, tz])
        
    def apply(self, volume):
        # Apply rotation and translation to the input volume
        return np.matmul(volume, self.R) + self.T


We are computing three separate rotation matrices (R_x, R_y, R_z) to represent the rotations around the x, y, and z axes, respectively. Each rotation matrix is responsible for a single rotation, and the final rotation matrix R is computed by multiplying the three rotation matrices in a specific order (R_zR_yR_x). This is known as the "Z-Y-X Euler angle convention", which means we rotate around the z-axis first, then the y-axis, and finally the x-axis.

This is a common way to represent a 3D rotation, where multiple rotations around different axes are combined into a single rotation matrix. By decomposing the rotation into three separate rotations, it becomes easier to specify the rotations in a more intuitive way, and it allows for more flexibility when applying the rotations.

In other words, the order of the rotations is important, and each rotation matrix is a representation of a rotation around a certain axis (x,y,z) and that is the reason we are computing three rotation matrixes.

The translation vector is computed by simply creating a 1-D array of the translation parameters tx, ty, and tz.
This vector is added to every point of the image volume after the rotation matrix has been applied.

The translation vector is simply the array [tx, ty, tz], which represents the translation along the x, y, and z axes, respectively in voxels. This vector is added to the final output of the transformed volume, the idea is that it shifts the entire transformed volume by the specified amount.

The translation vector is represented by a 1-D array because it is a vector quantity and it is a 1-D array because it only has one dimension and it is a vector of size 3(x,y,z) which corresponds to the 3D space.

## Class member function compute ddf that defines a 3D displacement vector

In [2]:
class RigidTransform:

    # Initialize class
    def __init__(self, rotations, translations):
        self.rx = rotations[0]
        self.ry = rotations[1]
        self.rz = rotations[2]
        self.tx = translations[0]
        self.ty = translations[1]
        self.tz = translations[2]

        # Precompute rotation matrices around the X, Y, and Z axis
        rot_x = np.array([[1, 0, 0],
                                 [0, np.cos(self.rx), -np.sin(self.rx)],
                                 [0, np.sin(self.rx), np.cos(self.rx)]])
                                 
        rot_y = np.array([[np.cos(self.ry), 0, np.sin(self.ry)],
                                [0, 1, 0],
                                [-np.sin(self.ry), 0, np.cos(self.ry)]])

        rot_z = np.array([[np.cos(self.rz), -np.sin(self.rz), 0],
                                [np.sin(self.rz), np.cos(self.rz), 0],
                                [0, 0, 1]])

        # Compose rotation matrix from yaw, pitch, and roll 
        # Correct order is Z, Y, X
        self.rot_vec = np.matmul(rot_z, np.matmul(rot_y, rot_x))

        # Compose translation vector
        self.trans_vec = np.array([self.tx, self.ty, self.tz])

        # Precompute the dense displacement field DDF
        self.ddf = self.compute_ddf((self.warped_image_size_x, self.warped_image_size_y, self.warped_image_size_z))

    # Implement a class member function compute_ddf that computes displacement vector
    # this code takes a point in the warped image,
    # undo the translation by subtracting the translation vector from the point,
    # then undo the rotation by applying the inverse rotation matrix, 
    # and the result is the displacement vector from the warped image to the original image
    #  at that voxel location.
    def compute_ddf(self, warped_image_size):
        # Compute displacement vector
        self.warped_image_size_x, self.warped_image_size_y, self.warped_image_size_z = warped_image_size
            # Pre-allocate displacement vector
        self.ddf = np.zeros((self.warped_image_size_x, self.warped_image_size_y, self.warped_image_size_z, 3))

        for x in range(self.warped_image_size_x):
            for y in range(self.warped_image_size_y):
                for z in range(self.warped_image_size_z):
                    # Compute displacement vector at each voxel
                    self.ddf[x, y, z, :] = np.matmul(np.linalg.inv(self.rot_vec), np.array([x, y, z]) - self.trans_vec)




        
        

In [3]:
a, b, c = (1, 2, 3); print(a, b, c)

1 2 3


In [2]:
np.zeros((1, 2, 3, 3))

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]])

The image coordinate system used in the above function is the standard Cartesian coordinate system, in which the origin is located at the center of the image, the x-axis is oriented from left to right, the y-axis is oriented from top to bottom, and the z-axis is oriented from front to back. The unit used is voxel, which means that the distance between two adjacent voxels is 1 in all three dimensions (x, y, z).

This coordinate system is consistent for both the warped image and the original image, meaning that the coordinates of the voxels in both images are defined in the same way. The origin of the coordinate system is located at the center of the image, which makes it easy to apply rotations and translations to the image while maintaining the same consistent coordinate system.

The orientation of the axis is also consistent, meaning that the x-axis is oriented from left to right, the y-axis is oriented from top to bottom, and the z-axis is oriented from front to back in both images. This makes it easy to understand the displacement vectors computed by the DDF, as the displacement values along each axis correspond to the standard x, y, and z axes.

In summary, the image coordinate system used in the above function is a standard Cartesian coordinate system with the origin located at the center of the image, the x-axis oriented from left to right, the y-axis oriented from top to bottom, and the z-axis oriented from front to back. The unit used is voxel, which is consistent for both the warped image and the original image.

## Class member function warp which returns a warped image volume in a numpy array

In [13]:
from scipy.interpolate import interpn
class RigidTransform:

    # Initialize class
    def __init__(self, rotations, translations, warped_image_size):
        self.rx = rotations[0]
        self.ry = rotations[1]
        self.rz = rotations[2]
        self.tx = translations[0]
        self.ty = translations[1]
        self.tz = translations[2]
        self.warped_image_size_x = warped_image_size[0]
        self.warped_image_size_y = warped_image_size[1]
        self.warped_image_size_z = warped_image_size[2]

        # Precompute rotation matrices around the X, Y, and Z axis
        rot_x = np.array([[1, 0, 0],
                                 [0, np.cos(self.rx), -np.sin(self.rx)],
                                 [0, np.sin(self.rx), np.cos(self.rx)]])
                                 
        rot_y = np.array([[np.cos(self.ry), 0, np.sin(self.ry)],
                                [0, 1, 0],
                                [-np.sin(self.ry), 0, np.cos(self.ry)]])

        rot_z = np.array([[np.cos(self.rz), -np.sin(self.rz), 0],
                                [np.sin(self.rz), np.cos(self.rz), 0],
                                [0, 0, 1]])

        # Compose rotation matrix from yaw, pitch, and roll 
        # Correct order is Z, Y, X
        self.rot_vec = np.matmul(rot_z, np.matmul(rot_y, rot_x))

        # Compose translation vector
        self.trans_vec = np.array([self.tx, self.ty, self.tz])

        # Precompute the dense displacement field DDF
        #self.ddf = self.compute_ddf((self.warped_image_size_x, self.warped_image_size_y, self.warped_image_size_z))
        self.compute_ddf((self.warped_image_size_x, self.warped_image_size_y, self.warped_image_size_z))

    # Implement a class member function compute_ddf that computes displacement vector
    # this code takes a point in the warped image,
    # undo the translation by subtracting the translation vector from the point,
    # then undo the rotation by applying the inverse rotation matrix, 
    # and the result is the displacement vector from the warped image to the original image
    #  at that voxel location.
    def compute_ddf(self, warped_image_size):
        # Compute displacement vector
        self.warped_image_size_x, self.warped_image_size_y, self.warped_image_size_z = warped_image_size
            # Pre-allocate displacement vector
        self.ddf = np.zeros((self.warped_image_size_x, self.warped_image_size_y, self.warped_image_size_z, 3))

        for x in range(self.warped_image_size_x):
            for y in range(self.warped_image_size_y):
                for z in range(self.warped_image_size_z):
                    # Compute displacement vector at each voxel
                    self.ddf[x, y, z, :] = np.matmul(np.linalg.inv(self.rot_vec), np.array([x, y, z]) - self.trans_vec)


    # Implement a class member function warp that returns a warped image volume in a Numpy Array
    def warp(self, volume):
        # Pre-allocate warped volume
        warped_volume = np.zeros_like(volume)
        for x in range(volume.shape[0]):
            for y in range(volume.shape[1]):
                for z in range(volume.shape[2]):
                    # Compute new voxle coordinates in the warped image
                    new_x, new_y, new_z = np.matmul(self.rot_vec, np.array([x, y, z])) + self.trans_vec

                    # Resample intensity values at the new coordinates
                    warped_volume[x, y, z] = interpn(np.array([x, y, z], ), volume, np.array((new_x, new_y, new_z),), method = 'linear', bounds_error = False, fill_value = None)
                    #warped_volume[x, y, z] = volume[new_x, new_y, new_z]

        return warped_volume

    # Implement a class member function compose which represents a combination of two transforms
    def compose(self, rotations2, translations2):
        # Compute the combined rotation matrix
        rot_x2 = np.array([[1, 0, 0],
                            [0, np.cos(rotations2[0]), -np.sin(rotations2[0])],
                            [0, np.sin(rotations2[0]), np.cos(rotations2[0])]])

        rot_y2 = np.array([[np.cos(rotations2[1]), 0, np.sin(rotations2[1])],
                            [0, 1, 0],
                            [-np.sin(rotations2[1]), 0, np.cos(rotations2[1])]])

        rot_z2 = np.array([[np.cos(rotations2[2]), -np.sin(rotations2[2]), 0],
                            [np.sin(rotations2[2]), np.cos(rotations2[2]), 0],
                            [0, 0, 1]])

        # Compose rotation matrix from yaw, pitch, and roll 
        # Correct order is Z, Y, X
        rot_vec2 = np.matmul(rot_z2, np.matmul(rot_y2, rot_x2))
        R_composed = np.matmul(self.rot_vec, rot_vec2)

        
        # Compute the combined translation vector
        #T_composed = np.matmul(self.rot_vec, np.array(translations2)) + self.trans_vec
        T_composed = np.matmul(self.rot_vec, self.trans_vec) + np.array(translations2)

        # Create a new RigidTransform object with the composed transformations
        composed_transform = RigidTransform(R_composed, T_composed)

        # Update the displacement field
        composed_transform.compute_ddf((self.warped_image_size_x, self.warped_image_size_y, self.warped_image_size_z))

        return composed_transform
        
        
                    

### Test the composed transformations

Manually define the ranges of translation and rotation parameters, according to the coordinate system defined in `RigidTransform` class.

-1000 to 1000 voxels which allows for translations up to 1000 voxels in any direction. For rotation, -180 to 180, to allow for rotations of up to 180 degrees in any direction


In [14]:
# Define translation and rotation parameters
rotations = np.array([30, 90, 150])*np.pi/180
translations = np.array([10, 20, 30])

## Instantiate an object that represent a transformation

In [15]:
rigid_transform = RigidTransform(rotations = rotations, translations = translations, warped_image_size = (100, 100, 100))